# Week 3: Validating Data and Predictive Pipelines
#### Making Meaningful Predictions from Data

This week we were introduced to the concept of validation and pipelines. In this notebook, we will go over basic training, testing, and validating data and move on to some pipeline practice as shown in lecture videos.

---

## Part 1: Training, Validating, Testing
1. **Training datasets** form the basis of a model's learning, teaching it how to estimate its output.
2. **Validation datasets** check the accuracy, quality, and integrity of a model, as well as fine-tuning any parameters that are not directly optimized.
3. **Test datasets** are usually what we are most interested in -- how does the model perform with previously unseen data that reflects real-world cases?

To learn more about the difference between these, click here: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7

A good model is one that generalizes to new data! If a model performs well on a training set but not on new data, then it is probably **overfitting** the training data.

#### "Theorems" about these sets

* The training error increases as lambda increases
* The validation and test error are at least as large as the training error (assuming infinitely large random partitions)
* The validation/test error will usually have a “sweet spot” between under- and over-fitting

### The Data

Unzip the `reviews.json` file in the Week 2 folder. This dataset contains 25,000 Yelp reviews of various businesses. For this notebook, we will be looking at how well various words can predict a business' rating.

### Reading the Data
First we import a few libraries. Most of these are the same as before, though a few new libraries are included for string processing. 

Then, specify the path of the file. You may need to change the given path according to your local environment. This should be familiar if you took Course 1 (*Basic Data Ingestion, Processing, and Visualization*) of the Python Data Products specialization already.

In [ ]:
import json
from collections import defaultdict
import string # Some string utilities
import random
from nltk.stem.porter import PorterStemmer # Stemming
import numpy

In [ ]:
path = "./datasets/reviews.json"
f = open(path, 'rt', encoding="utf8")

# Load in JSON file
dataset = []
for i in range(25000):
    dataset.append(json.loads(f.readline()))

In [ ]:
# Look at first entry in dataset
dataset[0]

### 1a. Examine Our Dataset
#### Counting Number of Unique Words
Before we start analyzing this data, let's see how many unique words we have.

In [ ]:
# Count total unique words
wordCount = defaultdict(int)
for d in dataset:
    for w in d['text'].split():
        wordCount[w] += 1

print(len(wordCount))

That's a lot of words! This number of words is too many to deal with (i.e., it would result in a 119,350 dimensional feature vector if used naively). Let's try to reduce this by removing punctuation and capitalization, so that two instances of the same word will be counted as being the same even if punctuated or capitalized differently.

In [ ]:
# Count total unique words, ignoring punctuation and capitalization
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

print(len(wordCount))

We're still left with a large number of words, so possibly we can reduce this number of words further if we treat different word inflections (e.g. "drinks" vs. "drinking") as being instances of the same word, by identifying their word stem (i.e., "drink"). This process is called "stemming".
We perform stemming using a stemmer from the NLTK (Natural Language Toolkit) library.

In [ ]:
# Count total unique words, ignoring punctuation and capitalization and using stemming.
# This one might take a bit of time to run -- be patient!
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer() # object doing the stemming
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w) # with stemming
        wordCount[w] += 1

print(len(wordCount))

#### Extracting and Building Features from the Most Common Words
That's still a lot of words. One way to deal with this is to take the most common words and build features out of those.

First we build a few data structures to count the number of instances of each word. Here we remove punctuation and capitalization, but do not apply stemming.

In [ ]:
# Count total unique words, ignoring punctuation and capitalization
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

Having counted the number of instances of each word, we can sort them to find the most common, and build a word index based on these frequencies. For example, the most frequent word will have index 0, the secont most frequent will have index 1, etc.

In [ ]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse() # Most frequent = index 0

words = [x[1] for x in counts[:1000]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

#### Building our Bag-of-Words Feature Representation
Now that we have our dictionary of common words, we can now build a feature vector by counting how often each of these words appears in each review. This is called a **"bag-of-words" feature representation**. This results in a 1,000 dimensional feature vector (1,001 if we include an offset term).

In [ ]:
# Function for building bag-of-words
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

### 1b. Build Training Dataset
We will build our training set by doing the following:
1. Shuffle entries
2. Define `X` - matrix of features
3. Define `y` - the outcomes
4. Perform least squares regression

In [ ]:
random.shuffle(dataset)

In [ ]:
X = [feature(d) for d in dataset] # using the bag-of-words function above

In [ ]:
y = [d['stars'] for d in dataset]

In [ ]:
theta,residuals,rank,s = numpy.linalg.lstsq(X, y,rcond=None)

### 1c. Visualize Important Words
Once the model has finished training, we can examine which are the most positive (or negative) words by looking at the largest (or smallest) corresponding values for theta To do so, let's sort our words based on their corresponding weights from `theta`:

In [ ]:
wordWeights = list(zip(theta, words + ['offset']))
wordWeights.sort()

In [ ]:
# 10 most negative words
wordWeights[:10]

In [ ]:
# 9 most positive words (10th is offset term)
wordWeights[-10:]

### 1d. Include Regularizer
**Regularization** is the process of penalizing model complexity during training. In this case, though the above model was effective, it was also very high dimensional, and thus may have been prone to **overfitting**. We can try to address this by adding a regularizer to our model.

The "Ridge" class from `sklearn` implements a least squares regression model (as in our example above) that includes a regularizer. The strength of the regularizer is controlled by the parameter `alpha` (equivalent to `lambda` in the course lectures). Otherwise, fitting the ridge regression model is exactly the same as fitting a regular least squares model! 

Here is the method call we are using: `model = linear_model.Ridge(1.0, fit_intercept=False)`

**Note the two extra parameters:** The first is the regularization strength (`alpha`). The second indicates we do not want the model to fit an intercept (since our feature vector already includes one).

In [ ]:
from sklearn import linear_model

In [ ]:
help(linear_model.Ridge)

In [ ]:
model = linear_model.Ridge(1.0, fit_intercept=False)
model.fit(X, y)

Let's examine the coefficients learned by our new regularized model.

In [ ]:
theta = model.coef_

In [ ]:
wordWeights = list(zip(theta, words + ['offset']))
wordWeights.sort()

In [ ]:
# 10 most negative words
wordWeights[:10]

In [ ]:
# 9 most positive words (10th is offset term)
wordWeights[-10:]

---
## Part 2: Regression Diagnostics: MSE and R^2 Review
Let's step away from programming for a moment. How do we evaluate our regressors? This section should be familiar from Week 1 of this course.

### 2a. MSE (Mean Squared Error)
What is the MSE (which we have already been using) and its relationship to the R^2 statistic? We start by extracting the predictions from our model and computing their squared differences:

In [ ]:
predictions = model.predict(X)

In [ ]:
differences = [(x-y)**2 for (x,y) in zip(predictions,y)]

The **MSE** is just the average (mean) of these squared differences, which we can compute with a few trivial Python commands:

In [ ]:
MSE = sum(differences) / len(differences)
print("MSE = " + str(MSE))

### 2b. R^2
As we saw in the lectures, the **R^2** (and the **FVU**, or "Fraction of Variance Unexplained") normalize the **Mean Squared Error** based on the variance of the data:

In [ ]:
FVU = MSE / numpy.var(y)
R2 = 1 - FVU
print("R2 = " + str(R2))

---

## Part 3: Predictive Pipelines

In its most basic form, a pipeline links steps together. In machine learning and data science, pipelines allow you transform data from one representation to another through a series of steps. 

A lower-level example can be found in Unix, where you can *pipeline* the *output* of some command as an *input* some other command with the `|` (e.g. `cat words.txt | wc -c` will result in the number of characters in `words.txt`). In this notebook, we will implement a regularization pipeline to help us train, test, and validate data, where (as stated before) **regularization** is the process of penalizing model complexity during training.

Let's import libraries and read our dataset in again.

In [ ]:
import json
from collections import defaultdict
import string
import random

In [ ]:
path = "./datasets/reviews.json"
f = open(path, 'rt', encoding="utf8")

# Load in JSON file
dataset = []
for i in range(25000):
    dataset.append(json.loads(f.readline()))

### 3a. Rebuild Common Word Features
This is identical to what we did in Part 1.

In [ ]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)

for d in dataset:
  r = ''.join([c for c in d['text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse() # Highest frequency = index 0

words = [x[1] for x in counts[:1000]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [ ]:
# Bag-of-words function
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

### 3b. Build the Validation Set
Again we split our data, but this time we split it into _three_ parts - a training, a validation, and a test component. Recall that validation sets check the accuracy, quality, and integrity of a model, as well as fine-tuning any parameters that are not directly optimized.

Let's shuffle the dataset and define `X` and `y` again.

In [ ]:
random.shuffle(dataset)
X = [feature(d) for d in dataset]
y = [d['stars'] for d in dataset]

In this notebook we'll use half of our data (and labels) for **training**, the next quarter for **validation**, and the final quarter for **testing**.

In [ ]:
N = len(X)

X_train = X[:N//2]
X_valid = X[N//2:3*N//4]
X_test = X[3*N//4:]

y_train = y[:N//2]
y_valid = y[N//2:3*N//4]
y_test = y[3*N//4:]

In [ ]:
# Examine size of data
len(X), len(X_train), len(X_valid), len(X_test)

### 3c. Train the Model
Again we'll train a model based on regularized (Ridge) regression. Our MSE function is the same as before, but this time for convience takes an (already trained) model as an input parameter.

#### Finally, to implement the pipeline, we:
1. Iterate through various values of lambda
2. Fit a ridge regression model for each of these values
3. Evaluate the performance of this model on the validation set
4. Keep track of which model is the best we've seen so far (on the validation set)

In [ ]:
from sklearn import linear_model

In [ ]:
# Function to find Mean Squared Error
def MSE(model, X, y):
    predictions = model.predict(X)
    differences = [(a-b)**2 for (a,b) in zip(predictions, y)]
    return sum(differences) / len(differences)

In [ ]:
# Variables to keep track of the current best model and MSE
bestModel = None 
bestMSE = None

In [ ]:
# Train and fine-tune model with training and validation sets
for lamb in [0.01, 0.1, 1, 10, 100]:
    model = linear_model.Ridge(lamb, fit_intercept=False)
    model.fit(X_train, y_train)
    
    mseTrain = MSE(model, X_train, y_train)
    mseValid = MSE(model, X_valid, y_valid)
    
    print("lambda = " + str(lamb) + ", training/validation error = " +
          str(mseTrain) + '/' + str(mseValid))
    if not bestModel or mseValid < bestMSE:
        bestModel = model
        bestMSE = mseValid

Finally, we evaluate the performance of our best model on the **test** set. Note that this is the only time throughout the entire pipeline that we examine the test data!

In [ ]:
mseTest = MSE(bestModel, X_test, y_test)
print("test error = " + str(mseTest))

---

## You're all done!
You should be familiar with the basics of training, validating, and testing in Python by now. We encourage you to explore further by using your own datasets and thinking about research questions you can answer with pipelines. You will have a chance to show off your skills at the end of this course with your very own project!